## Dragon Real Estate Price Predictor


In [1]:
import pandas as pd


In [2]:
housing = pd.read_csv("data.csv")
housing.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [3]:
housing["CHAS"].value_counts()

0    471
1     35
Name: CHAS, dtype: int64

In [4]:
housing.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,501.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284341,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.705587,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.884000,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208000,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.625000,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [5]:
# %matplotlib inline
# import matplotlib.pyplot as plt
# housing.hist(bins=50, figsize=(20,15))
# plt.show() 

## Train test splitting



In [6]:
import numpy as np
# manual training and testing
def split_train_test(data, test_ratio):
    np.random.seed(42)      #  this ensures that random values are fixed and not change every time
    shuffled=np.random.permutation(len(data))
    test_set_size=int(len(data)*test_ratio)
    test_indices=shuffled[:20]
    train_indices=shuffled[20:]
    return data.iloc[train_indices], data.iloc[test_indices]

# train_set, test_set = split_train_test(housing, 0.2)    

In [7]:
# training and testing by help of inbuilt fn
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)


# here there is a problem, for eg CHAS hos only two values 0 and 1, 0-475 and 1-35
# but in spliting train and test data suppose traindata = 402, testdata=104
# if there are no 1's in train data then our program forms wrong pattern that 
# there is only one possibility of CHAS i.e, 1

In [8]:
# to solve above problem
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing['CHAS']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [9]:
strat_test_set['CHAS'].value_counts()

0    95
1     7
Name: CHAS, dtype: int64

In [10]:
strat_train_set['CHAS'].value_counts()

0    376
1     28
Name: CHAS, dtype: int64

In [11]:
# 95/7 ~= 376/28

In [12]:
housing = strat_train_set.copy() #this is to be done for large data and not include test data in it

## Looking for correlations

In [13]:
corr_matrix = housing.corr()
corr_matrix['MEDV'].sort_values(ascending=False)

MEDV       1.000000
RM         0.680857
B          0.361761
ZN         0.339741
DIS        0.240451
CHAS       0.205066
AGE       -0.364596
RAD       -0.374693
CRIM      -0.393715
NOX       -0.422873
TAX       -0.456657
INDUS     -0.473516
PTRATIO   -0.493534
LSTAT     -0.740494
Name: MEDV, dtype: float64

In [14]:
# if value is 1 meaning strong positive correlation
# if value is -1 meaning strong negative correlation
# next Rm value is 0.69 which is high positive correlation which means if RM increases chances
# of increasing MEDV increases, then ZN and B are weak positive correlation
# similarly lstat is high neg corr, lesser value of lstat higher value of medv

In [15]:
# from pandas.plotting import scatter_matrix
# attributes = ["MEDV", "RM","ZN","LSTAT"]
# scatter_matrix(housing[attributes], figsize=(12,8))

In [16]:
# housing.plot(kind="scatter", x="RM",y="MEDV",alpha=0.8)

## Trying out Attribute combinations

In [17]:
housing["TAXRM"]=housing["TAX"]/housing["RM"] #you can try any combination
corr_matrix = housing.corr()
corr_matrix['MEDV'].sort_values(ascending=False)
# finally we are not adding this in our main data for now........

MEDV       1.000000
RM         0.680857
B          0.361761
ZN         0.339741
DIS        0.240451
CHAS       0.205066
AGE       -0.364596
RAD       -0.374693
CRIM      -0.393715
NOX       -0.422873
TAX       -0.456657
INDUS     -0.473516
PTRATIO   -0.493534
TAXRM     -0.528626
LSTAT     -0.740494
Name: MEDV, dtype: float64

In [18]:
# housing.plot(kind="scatter", x="TAXRM",y="MEDV",alpha=0.8)

In [19]:
housing = strat_train_set.drop("MEDV", axis=1)
housing_labels = strat_train_set["MEDV"].copy()

## Missing attributes

In [20]:
# To take care of missing attributes, you have three options:
#     1. Get rid of the missing data points
#     2. Get rid of the whole attribute
#     3. Set the value to some value(0, mean or median)

In [21]:
a=housing.dropna(subset=["RM"]) # option 1
a.shape

(399, 13)

In [22]:
housing.drop("RM", axis=1).shape  #option 2
# Note that there is no RM column and also note that the original housing dataframe will remain unchanged

(404, 12)

In [23]:
median = housing["RM"].median()  #option 3
median

6.209

In [24]:
housing["RM"].fillna(median)
# Note that the original housing dataframe will remain unchanged

254    6.108
348    6.635
476    6.484
321    6.376
326    6.312
       ...  
155    6.152
423    6.103
98     7.820
455    6.525
216    5.888
Name: RM, Length: 404, dtype: float64

In [25]:
housing.describe()    # before we started filling missing attributes

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
count,404.000000,404.000000,404.000000,404.000000,404.000000,399.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000
mean,3.602814,10.836634,11.344950,0.069307,0.558064,6.279481,69.039851,3.746210,9.735149,412.341584,18.473267,353.392822,12.791609
std,8.099383,22.150636,6.877817,0.254290,0.116875,0.716784,28.258248,2.099057,8.731259,168.672623,2.129243,96.069235,7.235740
min,0.006320,0.000000,0.740000,0.000000,0.389000,3.561000,2.900000,1.129600,1.000000,187.000000,13.000000,0.320000,1.730000
25%,0.086962,0.000000,5.190000,0.000000,0.453000,5.876500,44.850000,2.035975,4.000000,284.000000,17.400000,374.617500,6.847500
50%,0.286735,0.000000,9.900000,0.000000,0.538000,6.209000,78.200000,3.122200,5.000000,337.000000,19.000000,390.955000,11.570000
75%,3.731923,12.500000,18.100000,0.000000,0.631000,6.630500,94.100000,5.100400,24.000000,666.000000,20.200000,395.630000,17.102500
max,73.534100,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,36.980000


In [26]:
#for reflecting the changes in original train and test data i.e, null alues to median
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
imputer.fit(housing)

SimpleImputer(strategy='median')

In [27]:
imputer.statistics_    # calculates for every row and automatically fills

array([2.86735e-01, 0.00000e+00, 9.90000e+00, 0.00000e+00, 5.38000e-01,
       6.20900e+00, 7.82000e+01, 3.12220e+00, 5.00000e+00, 3.37000e+02,
       1.90000e+01, 3.90955e+02, 1.15700e+01])

In [28]:
X = imputer.transform(housing)

In [29]:
housing_tr = pd.DataFrame(X, columns=housing.columns)

In [30]:
housing_tr.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
count,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000
mean,3.602814,10.836634,11.344950,0.069307,0.558064,6.278609,69.039851,3.746210,9.735149,412.341584,18.473267,353.392822,12.791609
std,8.099383,22.150636,6.877817,0.254290,0.116875,0.712366,28.258248,2.099057,8.731259,168.672623,2.129243,96.069235,7.235740
min,0.006320,0.000000,0.740000,0.000000,0.389000,3.561000,2.900000,1.129600,1.000000,187.000000,13.000000,0.320000,1.730000
25%,0.086962,0.000000,5.190000,0.000000,0.453000,5.878750,44.850000,2.035975,4.000000,284.000000,17.400000,374.617500,6.847500
50%,0.286735,0.000000,9.900000,0.000000,0.538000,6.209000,78.200000,3.122200,5.000000,337.000000,19.000000,390.955000,11.570000
75%,3.731923,12.500000,18.100000,0.000000,0.631000,6.630000,94.100000,5.100400,24.000000,666.000000,20.200000,395.630000,17.102500
max,73.534100,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,36.980000


## Scikit learn Design

Primarily, three types of objects

1) Estimators - It estimates some parameter based on a dataset. 
Eg. imputer. It has a fit method and transform method. 
Fit method - Fits the dataset and calculates internal parameters

2) Transformers - transform method takes input and returns output based on the learnings from fit(). 
It also has a convenience function called fit_transform() which fits and then transforms.

3) Predictors - LinearRegression model is an example of predictor. 
fit() and predict() are two common functions. It also gives score() function which will 
evaluate the predictions.

## Feature Scaling


Primarily, two types of feature scaling methods:

1) Min-max scaling (Normalization) (value - min)/(max - min) 
Sklearn provides a class called MinMaxScaler for this

2) Standardization (value - mean)/std 
Sklearn provides a class called StandardScaler for this

## Creating a pipeline

In [31]:
# instead of doing imputer, directly you can opt for pipeline which automates things
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
my_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
#     add as many as you want...
    ('std_scaler',StandardScaler())
])


In [32]:
housing_num_tr = my_pipeline.fit_transform(housing)
housing_num_tr    # this is a numpy array

array([[-0.43942006,  3.12628155, -1.12165014, ..., -0.97491834,
         0.41164221, -0.86091034],
       [-0.44352175,  3.12628155, -1.35893781, ..., -0.69277865,
         0.39131918, -0.94116739],
       [ 0.15682292, -0.4898311 ,  0.98336806, ...,  0.81196637,
         0.44624347,  0.81480158],
       ...,
       [-0.43525657, -0.4898311 , -1.23083158, ..., -0.22254583,
         0.41831233, -1.27603303],
       [ 0.14210728, -0.4898311 ,  0.98336806, ...,  0.81196637,
        -3.15239177,  0.73869575],
       [-0.43974024, -0.4898311 ,  0.37049623, ..., -0.97491834,
         0.41070422,  0.09940681]])

## Selecting a desired model for the problem

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
# model = LinearRegression()
# model = DecisionTreeRegressor()    # this model is bad becouse mse=0, which means it is overfitting
model = RandomForestRegressor()
model.fit(housing_num_tr, housing_labels)
housing_num_tr.shape

(404, 13)

In [34]:
some_data = housing.iloc[:5]

In [35]:
some_labels = housing_labels.iloc[:5]

In [36]:
prepared_data = my_pipeline.transform(some_data)

In [37]:
model.predict(prepared_data)

array([22.267, 25.37 , 16.486, 23.219, 23.49 ])

In [38]:
np.array(some_labels)

array([21.9, 24.5, 16.7, 23.1, 23. ])

## Evaluating the model

In [39]:
from sklearn.metrics import mean_squared_error
housing_predictions = model.predict(housing_num_tr)
mse = mean_squared_error(housing_labels, housing_predictions)
rmse = np.sqrt(mse)

In [40]:
rmse

1.2276959715315716

## Using better evaluation techniques - Cross Validation

In [41]:
# for eg 1 2 3 4 5 6 7 8 9, firstly it will train expect 1 and check for 1, then repeat it for 2, 3..
# so on.. to last value
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, housing_num_tr, housing_labels, scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)

In [42]:
rmse_scores

array([2.88881783, 2.8528994 , 4.41985373, 2.6308078 , 3.76744051,
       2.75213552, 4.79458005, 3.24334951, 3.04242963, 3.24737381])

In [43]:
def print_scores(scores):
    print("Scores:", scores)
    print("Mean: ", scores.mean())
    print("Standard deviation: ", scores.std())

In [44]:
print_scores(rmse_scores)

Scores: [2.88881783 2.8528994  4.41985373 2.6308078  3.76744051 2.75213552
 4.79458005 3.24334951 3.04242963 3.24737381]
Mean:  3.3639687779476284
Standard deviation:  0.6967558527716741


## Saving the model

In [45]:
from joblib import dump, load
dump(model, 'Dragon.joblib') 

['Dragon.joblib']

## Testing the model on test data

In [49]:
X_test = strat_test_set.drop("MEDV", axis=1)
Y_test = strat_test_set["MEDV"].copy()
X_test_prepared = my_pipeline.transform(X_test)
final_predictions = model.predict(X_test_prepared)
final_mse = mean_squared_error(Y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
print(final_predictions, list(Y_test))

[24.531 11.125 25.427 21.913 18.336 14.475 20.023 14.347 31.69  40.892
 19.675 11.789 23.736 28.184 19.697 10.861 31.781 14.096 23.508 18.418
 20.129 17.787 17.86  21.814 18.903 31.366 16.073 32.563  8.845 33.334
 23.763 21.237 22.71  10.423 21.372 11.084 43.091 24.475 23.173 42.953
 23.881 30.312 20.459 20.856 18.873 32.974 44.218 20.144 20.445 21.569
 21.971 14.503 21.271 14.943 25.098 32.65  41.663 29.308 19.055 20.936
 46.438  9.819 19.075 25.037 14.679 32.832 20.321 18.294 19.409 34.498
 26.324 22.905 21.256 22.386 34.526 13.005 15.47  20.126 20.831 21.477
 22.128 21.044 14.425 23.063 20.72  21.281 14.362 21.187 21.451 23.358
 18.954 26.664  7.291 26.972 19.364 29.725 20.28  31.465 14.708 26.789
 21.395 20.323] [16.5, 10.2, 30.1, 23.0, 14.4, 15.6, 19.4, 14.1, 30.3, 35.2, 23.1, 13.8, 25.0, 27.9, 19.5, 12.3, 32.2, 13.5, 23.8, 21.7, 19.2, 19.5, 10.4, 23.2, 18.6, 28.5, 15.2, 32.0, 7.2, 34.6, 20.1, 20.6, 23.6, 13.1, 23.8, 12.7, 43.1, 24.7, 22.2, 44.0, 28.1, 31.0, 21.7, 23.4, 19.5, 33.1

In [47]:
final_rmse

2.9365060487780035

In [48]:
prepared_data[0]

array([-0.43942006,  3.12628155, -1.12165014, -0.27288841, -1.42262747,
       -0.23979304, -1.31238772,  2.61111401, -1.0016859 , -0.5778192 ,
       -0.97491834,  0.41164221, -0.86091034])

## Using the model

In [50]:
from joblib import dump, load
import numpy as np
model = load('Dragon.joblib') 
features = np.array([[-5.43942006, 4.12628155, -1.6165014, -0.67288841, -1.42262747,
       -11.44443979304, -49.31238772,  7.61111401, -26.0016879 , -0.5778192 ,
       -0.97491834,  0.41164221, -66.86091034]])
model.predict(features)

array([24.49])